In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm
import shap
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from  sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
import os
working_directory = os.getcwd()
print(working_directory)

/Users/ilinamitra


In [3]:
path = working_directory + '/Documents/df_updated.csv'
df = pd.read_csv(path, encoding='latin-1')
df.head()

,County,County.Code,suicides,Population,Crude_Rate,Uninsured: %,Mental_Health_Providers,Social_Associations,vet_pop_2010,Opioid Dispensing Rate per 100,...,in_labor_force,civilian_labot_force,employed,unemployed,armed_force,not_labor_force,total_occupied_housing,owner_occupied,renter_occupied,median_gross_rent
0,"Baldwin County, AL",1003,51,229287,22.242866,10.9,228,226,19614,65.0,...,99495,99317,95091,4226,178,72802,80930,60896,20034,1020
1,"Jefferson County, AL",1073,83,655342,12.665143,10.0,1379,953,42503,139.0,...,326672,326101,304994,21107,571,198562,261231,163799,97432,900
2,"Lee County, AL",1081,21,166831,12.587589,10.5,178,144,11532,35.4,...,79457,78849,74770,4079,608,50796,60237,37111,23126,856
3,"Madison County, AL",1089,54,379453,14.231011,9.9,613,403,33657,87.3,...,187581,186248,176910,9338,1333,104491,148189,99046,49143,857
4,"Mobile County, AL",1097,55,412716,13.326355,12.0,441,463,30347,98.0,...,186097,185327,174519,10808,770,141286,156251,100853,55398,853


In [4]:
df = df.drop(["Population","County", "County.Code","Year", "suicides","StateAbbr", "StateDesc", "LocationName", 
              "State", "...1"],axis=1)

In [5]:
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.8
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
#print(to_drop)

# Drop features with correlation greater than 0.8
df.drop(to_drop, axis=1, inplace=True)

In [6]:
df.shape

(447, 27)

In [7]:
df.columns

Index(['Crude_Rate', 'Uninsured: %', 'Mental_Health_Providers',
       'Opioid Dispensing Rate per 100', 'homicide_rate', 'RUCC_2013',
       'avg_temp', 'precip', 'Obesity among adults aged >=18 years',
       'Stroke among adults aged >=18 years',
       'Current asthma among adults aged >=18 years',
       'Depression among adults aged >=18 years',
       'Arthritis among adults aged >=18 years',
       'Binge drinking among adults aged >=18 years',
       'Current smoking among adults aged >=18 years',
       'Mental health not good for >=14 days among adults aged >=18 years',
       'Cholesterol screening among adults aged >=18 years',
       'Visits to doctor for routine checkup within the past year among adults aged >=18 years',
       'High cholesterol among adults aged >=18 years who have been screened in the past 5 years',
       'dem_perc', 'Score', 'Gun_Death_Rate_100k', 'black_pop', 'native_pop',
       'islander_pop', 'median_household_income', 'armed_force'],
      dtype

In [10]:
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)

imputer = imputer.fit(df[['homicide_rate']])
df['homicide_rate'] = imputer.transform(df[['homicide_rate']])

imputer = imputer.fit(df[['Obesity among adults aged >=18 years']])
df['Obesity among adults aged >=18 years'] = imputer.transform(
    df[['Obesity among adults aged >=18 years']])

imputer = imputer.fit(df[['Stroke among adults aged >=18 years']])
df['Stroke among adults aged >=18 years'] = imputer.transform(df[['Stroke among adults aged >=18 years']])

imputer = imputer.fit(df[['Current asthma among adults aged >=18 years']])
df['Current asthma among adults aged >=18 years'] = imputer.transform(
    df[['Current asthma among adults aged >=18 years']])

imputer = imputer.fit(df[['Depression among adults aged >=18 years']])
df['Depression among adults aged >=18 years'] = imputer.transform(
    df[['Depression among adults aged >=18 years']])

imputer = imputer.fit(df[['Arthritis among adults aged >=18 years']])
df['Arthritis among adults aged >=18 years'] = imputer.transform(
    df[['Arthritis among adults aged >=18 years']])

imputer = imputer.fit(df[['Binge drinking among adults aged >=18 years']])
df['Binge drinking among adults aged >=18 years'] = imputer.transform(
    df[['Binge drinking among adults aged >=18 years']])

imputer = imputer.fit(df[['Current smoking among adults aged >=18 years']])
df['Current smoking among adults aged >=18 years'] = imputer.transform(
    df[['Current smoking among adults aged >=18 years']])

imputer = imputer.fit(df[['Mental health not good for >=14 days among adults aged >=18 years']])
df['Mental health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Mental health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Cholesterol screening among adults aged >=18 years']])
df['Cholesterol screening among adults aged >=18 years'] = imputer.transform(
    df[['Cholesterol screening among adults aged >=18 years']])

imputer = imputer.fit(df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])
df['Visits to doctor for routine checkup within the past year among adults aged >=18 years'] = imputer.transform(
    df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])

imputer = imputer.fit(df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])
df['High cholesterol among adults aged >=18 years who have been screened in the past 5 years'] = imputer.transform(
    df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])

In [11]:
df.isnull().sum()

Crude_Rate                                                                                  0
Uninsured: %                                                                                0
Mental_Health_Providers                                                                     0
Opioid Dispensing Rate per 100                                                              0
homicide_rate                                                                               0
RUCC_2013                                                                                   0
avg_temp                                                                                    0
precip                                                                                      0
Obesity among adults aged >=18 years                                                        0
Stroke among adults aged >=18 years                                                         0
Current asthma among adults aged >=18 years                 

In [20]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (80/20)
    The R2 is maximized in cross-validation.
    '''
    
    
    random_state = 7
    
    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)
    #print('mean_xtrain': np.mean()) #df.mean() 
        
    #kfold split
    kf = KFold(n_splits=10,shuffle=True,random_state=24)
        
    # loop through the hyperparameter combinations or use GridSearchCV 
    grid = GridSearchCV(estimator = ML_algo, param_grid=param_grid,scoring = 'neg_mean_absolute_error', 
                            n_jobs = -1, cv = kf, return_train_score = True, verbose=True) 
        
    #fitting on training data
    grid.fit(X_train, y_train)
    results = pd.DataFrame(grid.cv_results_)

    print('best model parameters:',grid.best_params_)
    print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
    
    # calculate and save the test score
    y_test_pred = grid.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print(test_score)
        
    return grid, test_score

ML_algo = Lasso()
param_grid = {#'alpha':[0.01, 0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09, 0.10],
              'alpha':[0.0000000001, 0.00000000129, 0.0000000167, 0.000000215, 0.00000278, 0.0000359,
                      0.000464, 0.00599, 0.0774, 1],
             'max_iter': [100000]}

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

Fitting 10 folds for each of 10 candidates, totalling 100 fits
best model parameters: {'alpha': 0.0774, 'max_iter': 100000}
validation score: -2.6090702178211678
2.6458018547699926
2.6458018547699926


In [29]:
#having chosen the values from the param grid, running the model on 80% of data (training) and testing on 20% of data (test set)

y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (80/20)
    The R2 is maximized in cross-validation.
    '''
        
    random_state = 48

    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)


    model = ML_algo.fit(X_train, y_train)

    # calculate and save the test score
    y_test_pred = model.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print('test score:',test_score)
        
    return model, test_score

ML_algo = Lasso(alpha = 0.00599, max_iter = 100000)

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

test score: 2.3462290489730795
2.3462290489730795


In [30]:
#the sign of the coefficient usually stays the same but when the random state changes, the magnitude is subject to change
model.coef_

array([ 0.81207989, -0.54045981,  0.66124494, -0.57537035,  0.25576462,
       -0.48600898, -0.27505417, -1.91221197,  0.15139279,  0.83166646,
       -0.13219737,  1.91232922,  0.19324561,  0.70098725, -1.07352133,
        0.01583724, -1.79627318,  0.09530759, -1.06801848, -0.31775447,
        1.24496794, -0.19661189,  0.43709187,  0.08443329, -0.84578072,
        0.62536257])

In [28]:
pd.Series(model.coef_, index = X.columns)

Uninsured: %                                                                                0.921664
Mental_Health_Providers                                                                    -0.609523
Opioid Dispensing Rate per 100                                                              0.717549
homicide_rate                                                                              -0.336606
RUCC_2013                                                                                   0.185072
avg_temp                                                                                   -0.129306
precip                                                                                     -0.402706
Obesity among adults aged >=18 years                                                       -1.481009
Stroke among adults aged >=18 years                                                         0.000000
Current asthma among adults aged >=18 years                                                